# Bir gizli katmanla düzlemsel veri sınıflandırması

Gizli bir katmana sahip olacak ilk sinir ağınızı oluşturmanın zamanı geldi. Bu model ile lojistik regresyon kullanarak uyguladığınız model arasında büyük bir fark göreceksiniz.

**Nasıl yapacağınızı öğreneceksiniz:**
- Tek bir gizli katmana sahip 2 sınıflı bir sınıflandırma sinir ağı uygulayın
- Tanh gibi doğrusal olmayan aktivasyon fonksiyonu sahip birimleri kullanın
- Çapraz entropi kaybını hesaplayın
- İleri ve geri yayılımı uygulayın

## 1 - Paketler ##

Öncelikle bu çalışma dosyası sırasında ihtiyaç duyacağınız tüm paketleri import edelim.
- [numpy](www.numpy.org), Python ile bilimsel hesaplama için temel pakettir.
- [sklearn](http://scikit-learn.org/stable/) veri madenciliği ve veri analizi için basit ve verimli araçlar sağlar.
- [matplotlib](http://matplotlib.org), Python'da grafik çizmek için bir kütüphanedir.

In [ ]:
# Package imports
import numpy as np
import matplotlib.pyplot as plt
#from testCases import *
import sklearn
import sklearn.datasets
import sklearn.linear_model
#from planar_utils import plot_decision_boundary, sigmoid, load_planar_dataset, load_extra_datasets

%matplotlib inline

np.random.seed(1) # set a seed so that the results are consistent

In [ ]:
#@title yardımcı fonksiyonlar
def plot_decision_boundary(model, X, y):
    # Set min and max values and give it some padding
    x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
    y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole grid
    Z = model(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(X[0, :], X[1, :], c=y, cmap=plt.cm.Spectral)
    

def sigmoid(x):
    """
    Compute the sigmoid of x

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

def load_planar_dataset():
    np.random.seed(1)
    m = 400 # number of examples
    N = int(m/2) # number of points per class
    D = 2 # dimensionality
    X = np.zeros((m,D)) # data matrix where each row is a single example
    Y = np.zeros((m,1), dtype='uint8') # labels vector (0 for red, 1 for blue)
    a = 4 # maximum ray of the flower

    for j in range(2):
        ix = range(N*j,N*(j+1))
        t = np.linspace(j*3.12,(j+1)*3.12,N) + np.random.randn(N)*0.2 # theta
        r = a*np.sin(4*t) + np.random.randn(N)*0.2 # radius
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        Y[ix] = j
        
    X = X.T
    Y = Y.T

    return X, Y

def load_extra_datasets():  
    N = 200
    noisy_circles = sklearn.datasets.make_circles(n_samples=N, factor=.5, noise=.3)
    noisy_moons = sklearn.datasets.make_moons(n_samples=N, noise=.2)
    blobs = sklearn.datasets.make_blobs(n_samples=N, random_state=5, n_features=2, centers=6)
    gaussian_quantiles = sklearn.datasets.make_gaussian_quantiles(mean=None, cov=0.5, n_samples=N, n_features=2, n_classes=2, shuffle=True, random_state=None)
    no_structure = np.random.rand(N, 2), np.random.rand(N, 2)
    
    return noisy_circles, noisy_moons, blobs, gaussian_quantiles, no_structure

## 2 - Veri Kümesi ##

Öncelikle üzerinde çalışacağınız veri kümesini alalım. 
Aşağıdaki kod, `X` ve `Y` değişkenlerine "çiçek(flower)" 2 sınıfı bir veri kümesi yükleyecektir.

In [ ]:
X, Y = load_planar_dataset()

Matplotlib kullanarak veri kümesini görselleştirin. Veriler, biraz kırmızı (etiket y=0) ve biraz mavi (y=1) noktaları olan bir "çiçek(flower)" gibi görünüyor. Amacınız bu verilere uyacak bir model oluşturmaktır.

In [ ]:
# Visualize the data:
plt.scatter(X[0, :], X[1, :], c=Y, s=40, cmap=plt.cm.Spectral);

- özniteliklerini içeren bir sayısal dizi (matris) X (x1, x2)
- etiketlerinizi içeren bir sayısal dizi (vektör) Y (kırmızı:0, mavi:1).

Önce verilerimizin neye benzediğini daha iyi anlayalım.

**Alıştırma**: Kaç örneğiniz var? Ayrıca, "X" ve "Y" değişkenlerinin "boyutları" nedir?


In [ ]:
### START CODE HERE ### (≈ 3 lines of code)
shape_X = X.shape
shape_Y = Y.shape
m = Y.shape[1]  # training set size
### END CODE HERE ###

print ('The shape of X is: ' + str(shape_X))
print ('The shape of Y is: ' + str(shape_Y))
print ('I have m = %d training examples!' % (m))

**Beklenen Çıktı**:
       
<table style="width:20%">
  <tr>
    <td>shape of X</td>
    <td> (2, 400) </td> 
  </tr>
  <tr>
    <td>shape of Y</td>
    <td>(1, 400) </td> 
  </tr>
  <tr>
    <td>m</td>
    <td> 400 </td> 
  </tr>
</table>

## 3 - Basit Lojistik Regresyon

Tam bir sinir ağı kurmadan önce, lojistik regresyonun bu problem üzerinde nasıl performans gösterdiğini görelim. Bunu yapmak için sklearn'in yerleşik fonksiyonlarını kullanabilirsiniz. Veri kümesinde bir lojistik regresyon sınıflandırıcısı eğitmek için aşağıdaki kodu çalıştırın.

In [ ]:
# Train the logistic regression classifier
clf = sklearn.linear_model.LogisticRegressionCV();
clf.fit(X.T, Y.T);

Artık bu modellerin karar sınırlarını çizebilirsiniz. Aşağıdaki kodu çalıştırın.

In [ ]:
# Plot the decision boundary for logistic regression
plot_decision_boundary(lambda x: clf.predict(x), X, Y)
plt.title("Logistic Regression")

# Print accuracy
LR_predictions = clf.predict(X.T)
print ('Accuracy of logistic regression: %d ' % float((np.dot(Y, LR_predictions) + np.dot(1 - Y,1 - LR_predictions)) / float(Y.size) * 100) +
       '% ' + "(percentage of correctly labelled datapoints)")

**Beklenen Çıktı**:

<table style="width:20%">
  <tr>
    <td>Accuracy</td>
    <td> 47% </td> 
  </tr>
  
</table>


**Yorum**: Veri kümesi doğrusal olarak ayrılamaz, bu nedenle lojistik regresyon iyi performans göstermez. Umarım bir sinir ağı daha iyisini yapar. Şimdi bunu deneyelim!

## 4 - Sinir Ağı modeli

Lojistik regresyon, "çiçek veri kümesi (flower dataset)" üzerinde iyi çalışmadı. Şimdi tek bir gizli katmana sahip bir Sinir Ağı eğiteceksiniz.

**İşte modelimiz**:

<img src="https://drive.google.com/uc?id=1GG7lJxSkfx5cgS5sAX9BHV0Z_W_xcHpP" style="width:600px;height:300px;" alt="classification_kiank" title="classification_kiank">

**Matematiksel olarak**:

bir örnek için $x^{(i)}$:
$$z^{[1] (i)} =  W^{[1]} x^{(i)} + b^{[1] (i)}\tag{1}$$ 
$$a^{[1] (i)} = \tanh(z^{[1] (i)})\tag{2}$$
$$z^{[2] (i)} = W^{[2]} a^{[1] (i)} + b^{[2] (i)}\tag{3}$$
$$\hat{y}^{(i)} = a^{[2] (i)} = \sigma(z^{ [2] (i)})\tag{4}$$
$$y^{(i)}_{prediction} = \begin{cases} 1 & \mbox{if } a^{[2](i)} > 0.5 \\ 0 & \mbox{otherwise } \end{cases}\tag{5}$$

Tüm örneklerdeki tahminler göz önüne alındığında, $J$ maliyetini aşağıdaki gibi de hesaplayabilirsiniz:
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$

**Hatırlatma**: Bir Sinir Ağı oluşturmak için genel metodoloji şudur:
  1. Sinir ağı yapısını tanımlayın (giriş birimlerinin sayısı, gizli birimlerin sayısı, vb.).
  2. Modelin parametrelerini başlatın
  3. Döngü:
    - İleri yayılımı uygula
    - Hesaplama kaybı
    - Gradyanları elde etmek için geriye doğru yayılım uygulayın
    - Parametreleri güncelle (gradyan iniş)

Genellikle 1-3 arasındaki adımları hesaplamak için yardımcı fonksiyonlar oluşturursunuz ve ardından bunları `nn_model()` dediğimiz tek bir fonksiyonda birleştirirsiniz. `nn_model()`i oluşturup doğru parametreleri öğrendikten sonra, yeni veriler üzerinde tahminler yapabilirsiniz.

### 4.1 - Sinir ağı yapısını tanımlama ####

**Alıştırma**: Üç değişken tanımlayın:
  - n_x: giriş katmanının boyutu
  - n_h: gizli katmanın boyutu (bunu 4 olarak ayarlayın)
  - n_y: çıktı katmanının boyutu

**İpucu**: n_x ve n_y'yi bulmak için X ve Y şekillerini kullanın. Ayrıca, gizli katman boyutunu 4 olarak sabit kodlayın.

In [ ]:
#@title test_cases
#test_case
def layer_sizes_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(5, 3)
    Y_assess = np.random.randn(2, 3)
    return X_assess, Y_assess

def initialize_parameters_test_case():
    n_x, n_h, n_y = 2, 4, 1
    return n_x, n_h, n_y


def forward_propagation_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(2, 3)
    b1 = np.random.randn(4,1)
    b2 = np.array([[ -1.3]])

    parameters = {'W1': np.array([[-0.00416758, -0.00056267],
        [-0.02136196,  0.01640271],
        [-0.01793436, -0.00841747],
        [ 0.00502881, -0.01245288]]),
     'W2': np.array([[-0.01057952, -0.00909008,  0.00551454,  0.02292208]]),
     'b1': b1,
     'b2': b2}

    return X_assess, parameters

def compute_cost_test_case():
    np.random.seed(1)
    Y_assess = (np.random.randn(1, 3) > 0)
    parameters = {'W1': np.array([[-0.00416758, -0.00056267],
        [-0.02136196,  0.01640271],
        [-0.01793436, -0.00841747],
        [ 0.00502881, -0.01245288]]),
     'W2': np.array([[-0.01057952, -0.00909008,  0.00551454,  0.02292208]]),
     'b1': np.array([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.]]),
     'b2': np.array([[ 0.]])}

    a2 = (np.array([[ 0.5002307 ,  0.49985831,  0.50023963]]))
    
    return a2, Y_assess, parameters

def backward_propagation_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(2, 3)
    Y_assess = (np.random.randn(1, 3) > 0)
    parameters = {'W1': np.array([[-0.00416758, -0.00056267],
        [-0.02136196,  0.01640271],
        [-0.01793436, -0.00841747],
        [ 0.00502881, -0.01245288]]),
     'W2': np.array([[-0.01057952, -0.00909008,  0.00551454,  0.02292208]]),
     'b1': np.array([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.]]),
     'b2': np.array([[ 0.]])}

    cache = {'A1': np.array([[-0.00616578,  0.0020626 ,  0.00349619],
         [-0.05225116,  0.02725659, -0.02646251],
         [-0.02009721,  0.0036869 ,  0.02883756],
         [ 0.02152675, -0.01385234,  0.02599885]]),
  'A2': np.array([[ 0.5002307 ,  0.49985831,  0.50023963]]),
  'Z1': np.array([[-0.00616586,  0.0020626 ,  0.0034962 ],
         [-0.05229879,  0.02726335, -0.02646869],
         [-0.02009991,  0.00368692,  0.02884556],
         [ 0.02153007, -0.01385322,  0.02600471]]),
  'Z2': np.array([[ 0.00092281, -0.00056678,  0.00095853]])}
    return parameters, cache, X_assess, Y_assess

def update_parameters_test_case():
    parameters = {'W1': np.array([[-0.00615039,  0.0169021 ],
        [-0.02311792,  0.03137121],
        [-0.0169217 , -0.01752545],
        [ 0.00935436, -0.05018221]]),
 'W2': np.array([[-0.0104319 , -0.04019007,  0.01607211,  0.04440255]]),
 'b1': np.array([[ -8.97523455e-07],
        [  8.15562092e-06],
        [  6.04810633e-07],
        [ -2.54560700e-06]]),
 'b2': np.array([[  9.14954378e-05]])}

    grads = {'dW1': np.array([[ 0.00023322, -0.00205423],
        [ 0.00082222, -0.00700776],
        [-0.00031831,  0.0028636 ],
        [-0.00092857,  0.00809933]]),
 'dW2': np.array([[ -1.75740039e-05,   3.70231337e-03,  -1.25683095e-03,
          -2.55715317e-03]]),
 'db1': np.array([[  1.05570087e-07],
        [ -3.81814487e-06],
        [ -1.90155145e-07],
        [  5.46467802e-07]]),
 'db2': np.array([[ -1.08923140e-05]])}
    return parameters, grads

def nn_model_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(2, 3)
    Y_assess = (np.random.randn(1, 3) > 0)
    return X_assess, Y_assess

def predict_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(2, 3)
    parameters = {'W1': np.array([[-0.00615039,  0.0169021 ],
        [-0.02311792,  0.03137121],
        [-0.0169217 , -0.01752545],
        [ 0.00935436, -0.05018221]]),
     'W2': np.array([[-0.0104319 , -0.04019007,  0.01607211,  0.04440255]]),
     'b1': np.array([[ -8.97523455e-07],
        [  8.15562092e-06],
        [  6.04810633e-07],
        [ -2.54560700e-06]]),
     'b2': np.array([[  9.14954378e-05]])}
    return parameters, X_assess


In [ ]:
# GRADED FUNCTION: layer_sizes

def layer_sizes(X, Y):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    ### START CODE HERE ### (≈ 3 lines of code)
    n_x = X.shape[0] # size of input layer
    n_h = 4
    n_y = Y.shape[0] # size of output layer
    ### END CODE HERE ###
    return (n_x, n_h, n_y)

In [ ]:
X_assess, Y_assess = layer_sizes_test_case()
(n_x, n_h, n_y) = layer_sizes(X_assess, Y_assess)
print("The size of the input layer is: n_x = " + str(n_x))
print("The size of the hidden layer is: n_h = " + str(n_h))
print("The size of the output layer is: n_y = " + str(n_y))

**Beklenen Çıktı** (bunlar ağınız için kullanacağınız boyutlar değildir, sadece az önce kodladığınız işlevi değerlendirmek için kullanılırlar).

<table style="width:20%">
  <tr>
    <td>n_x</td>
    <td> 5 </td> 
  </tr>
  <tr>
    <td>n_h</td>
    <td> 4 </td> 
  </tr>
  <tr>
    <td>n_y</td>
    <td> 2 </td> 
  </tr>
</table>

### 4.2 - Modelin parametrelerini sıfırlayın ####

**Alıştırma**: `initialize_parameters()` fonksiyonunu uygulayın.

**Talimatlar**:
- Parametrelerinizin boyutlarının doğru olduğundan emin olun. Gerekirse yukarıdaki sinir ağı şekline bakın.
- Ağırlık matrislerini rastgele değerlerle başlatacaksınız.
  - Bir şekil matrisini (a,b) rastgele başlatmak için `np.random.randn(a,b) * 0.01` kullanın.

- Önyargı vektörlerini (bias vectors) sıfır olarak başlatacaksınız.
  - (a,b) şeklindeki bir matrisi sıfırlarla başlatmak için `np.zeros((a,b))` kullanın.

In [ ]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(2) # we set up a seed so that your output matches ours although the initialization is random.
    
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros(shape=(n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros(shape=(n_y, 1))
    ### END CODE HERE ###
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
n_x, n_h, n_y = initialize_parameters_test_case()

parameters = initialize_parameters(n_x, n_h, n_y)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Beklenen Çıktı**:

<table style="width:90%">
  <tr>
    <td>W1</td>
    <td> [[-0.00416758 -0.00056267]
 [-0.02136196  0.01640271]
 [-0.01793436 -0.00841747]
 [ 0.00502881 -0.01245288]] </td> 
  </tr>
  <tr>
    <td>b1</td>
    <td> [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]] </td> 
  </tr>
  <tr>
    <td>W2</td>
    <td> [[-0.01057952 -0.00909008  0.00551454  0.02292208]]</td> 
  </tr>
  <tr>
    <td>b2</td>
    <td> [[ 0.]] </td> 
  </tr>
</table>

### 4.3 - Döngü ####

**Soru**: `forward_propagation()` öğesini uygulayın.

**Talimatlar**:
- Sınıflandırıcınızın matematiksel temsiline yukarıda bakın.
- `sigmoid()` fonksiyonunu kullanabilirsiniz. Yukarıda yardımcı fonksiyonlar içerisinde tanımlanmıştı.
- `np.tanh()` fonksiyonunu kullanabilirsiniz. Numpy kütüphanesinin bir parçasıdır.
- Uygulamanız gereken adımlar şunlardır:
     1. Her parametreyi "parametreler" sözlüğünden (`initialize_parameters()` çıktısıdır) `parameters[".."]` kullanarak alın.
     2. İleri Yayılımı uygulayın. 
$Z^{[1]}, A^{[1]}, Z^{[2]}$ ve $A^{[2]}$ 

(eğitimdeki tüm örneklerle ilgili tüm tahminlerinizin vektörü) hesaplayın Ayarlamak).
- Geri yayılımda ihtiyaç duyulan değerler "`cache`"de saklanır. "`cache`", geri yayılım işlevine bir girdi olarak verilecektir.


In [ ]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    ### END CODE HERE ###
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    ### START CODE HERE ### (≈ 4 lines of code)
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    ### END CODE HERE ###
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [ ]:
X_assess, parameters = forward_propagation_test_case()

A2, cache = forward_propagation(X_assess, parameters)

# Note: we use the mean here just to make sure that your output matches ours. 
print(np.mean(cache['Z1']), np.mean(cache['A1']), np.mean(cache['Z2']), np.mean(cache['A2']))

**Beklenen Çıktı**:
<table style="width:55%">
  <tr>
    <td> -0.000499755777742 -0.000496963353232 0.000438187450959 0.500109546852 </td> 
  </tr>
</table>

Artık her örnek için $a^{[2](i)}$ içeren $A^{[2]}$ hesapladığınıza göre, aşağıdaki gibi maliyet fonksiyonunu hesaplayabilirsiniz. :

$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large{(} \small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right) \large{)} \small\tag{13}$$

**Alıştırma**: $J$ maliyetinin değerini hesaplamak için `compute_cost()` öğesini uygulayın.

**Talimatlar**:
- Çapraz entropi kaybını uygulamanın birçok yolu vardır. Size yardımcı olmak için, nasıl uygulayacağımızı size veriyoruz
$- \sum\limits_{i=0}^{m}  y^{(i)}\log(a^{[2](i)})$:
```python
logprobs = np.multiply(np.log(A2),Y)
cost = - np.sum(logprobs)                # no need to use a for loop!
```

(`np.multiply()` ve ardından `np.sum()` veya doğrudan `np.dot()` kullanabilirsiniz).


In [ ]:
# GRADED FUNCTION: compute_cost

def compute_cost(A2, Y, parameters):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    parameters -- python dictionary containing your parameters W1, b1, W2 and b2
    
    Returns:
    cost -- cross-entropy cost given equation (13)
    """
    
    m = Y.shape[1] # number of example
    
    # Retrieve W1 and W2 from parameters
    ### START CODE HERE ### (≈ 2 lines of code)
    W1 = parameters['W1']
    W2 = parameters['W2']
    ### END CODE HERE ###
    
    # Compute the cross-entropy cost
    ### START CODE HERE ### (≈ 2 lines of code)
    logprobs = np.multiply(np.log(A2), Y) + np.multiply((1 - Y), np.log(1 - A2))
    cost = - np.sum(logprobs) / m
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)     # makes sure cost is the dimension we expect. 
                                # E.g., turns [[17]] into 17 
    assert(isinstance(cost, float))
    
    return cost

In [ ]:
A2, Y_assess, parameters = compute_cost_test_case()

print("cost = " + str(compute_cost(A2, Y_assess, parameters)))

**Beklenen Çıktı**:
<table style="width:20%">
  <tr>
    <td>cost</td>
    <td> 0.692919893776 </td> 
  </tr>
  
</table>

**Soru**: `backward_propagation()` fonksiyonunu uygulayın.

**Talimatlar**:
Geri yayılım, derin öğrenmede genellikle en zor (en matematiksel) kısımdır. Size yardımcı olmak için, geri yayılımla ilgili dersten bir slayt yine burada. Vektörleştirilmiş bir uygulama oluşturduğunuz için bu slaydın sağındaki altı denklemi kullanmak isteyeceksiniz.

<img src="https://drive.google.com/uc?id=1nbhCtgM93yCN5QYaCXe3osezFCVgeZIl" style="width:600px;height:300px;" alt="grad_summary" title="grad_summary">

<!--
$\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } = \frac{1}{m} (a^{[2](i)} - y^{(i)})$

$\frac{\partial \mathcal{J} }{ \partial W_2 } = \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } a^{[1] (i) T} $

$\frac{\partial \mathcal{J} }{ \partial b_2 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)}}}$

$\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} } =  W_2^T \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } * ( 1 - a^{[1] (i) 2}) $

$\frac{\partial \mathcal{J} }{ \partial W_1 } = \frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} }  X^T $

$\frac{\partial \mathcal{J} _i }{ \partial b_1 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)}}}$

- Note that $*$ denotes elementwise multiplication.
- The notation you will use is common in deep learning coding:
    - dW1 = $\frac{\partial \mathcal{J} }{ \partial W_1 }$
    - db1 = $\frac{\partial \mathcal{J} }{ \partial b_1 }$
    - dW2 = $\frac{\partial \mathcal{J} }{ \partial W_2 }$
    - db2 = $\frac{\partial \mathcal{J} }{ \partial b_2 }$
    
!-->

**İpuçları:**
dZ1'i hesaplamak için $g^{[1]'}(Z^{[1]})$ hesaplamanız gerekir. 
  $g^{[1]}(.)$ tanh aktivasyon işlevi olduğundan, $a = g^{[1]}(z)$ ise $g^{[1]'}(z) = 1- a^2$. 
`(1 - np.power(A1, 2))` kullanarak $g^{[1]'}(Z^{[1]})$'i hesaplayabilirsin.

In [ ]:
# GRADED FUNCTION: backward_propagation

def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    ### START CODE HERE ### (≈ 2 lines of code)
    W1 = parameters['W1']
    W2 = parameters['W2']
    ### END CODE HERE ###
        
    # Retrieve also A1 and A2 from dictionary "cache".
    ### START CODE HERE ### (≈ 2 lines of code)
    A1 = cache['A1']
    A2 = cache['A2']
    ### END CODE HERE ###
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    ### START CODE HERE ### (≈ 6 lines of code, corresponding to 6 equations on slide above)
    dZ2= A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [ ]:
parameters, cache, X_assess, Y_assess = backward_propagation_test_case()

grads = backward_propagation(parameters, cache, X_assess, Y_assess)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dW2 = "+ str(grads["dW2"]))
print ("db2 = "+ str(grads["db2"]))

**Beklenen Çıktı**:

<table style="width:80%">
  <tr>
    <td>dW1</td>
    <td> [[ 0.01018708 -0.00708701]
 [ 0.00873447 -0.0060768 ]
 [-0.00530847  0.00369379]
 [-0.02206365  0.01535126]] </td> 
  </tr>
  <tr>
    <td>db1</td>
    <td>  [[-0.00069728]
 [-0.00060606]
 [ 0.000364  ]
 [ 0.00151207]] </td> 
  </tr>
  <tr>
    <td>dW2</td>
    <td> [[ 0.00363613  0.03153604  0.01162914 -0.01318316]] </td> 
  </tr>
  <tr>
    <td>db2</td>
    <td> [[ 0.06589489]] </td> 
  </tr>
</table>  

**Soru**: Güncelleme kuralını uygulayın. Gradyan iniş kullanın. (W1, b1, W2, b2)'yi güncellemek için (dW1, db1, dW2, db2) kullanmanız gerekir.

**Genel gradyan iniş kuralı**: $ \theta = \theta - \alpha \frac{\partial J }{ \partial \theta }$ burada $\alpha$ öğrenme oranıdır ve $\theta$ bir parametreyi temsil eder.

**Resim**: İyi bir öğrenme oranına (yakınsayan) ve kötü bir öğrenme oranına (uzaklaşan) sahip gradyan iniş algoritması.

<img src="https://drive.google.com/uc?id=1zEB5wdgCLh7KTKTEypMKr0czFIKoX5F_" style="width:650px;height:400px;" alt="sgd" title="sgd">



In [ ]:
# GRADED FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate=1.2):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    ### END CODE HERE ###
    
    # Retrieve each gradient from the dictionary "grads"
    ### START CODE HERE ### (≈ 4 lines of code)
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    ## END CODE HERE ###
    
    # Update rule for each parameter
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Beklenen Çıktı**:


<table style="width:80%">
  <tr>
    <td>W1</td>
    <td> [[-0.00643025  0.01936718]
 [-0.02410458  0.03978052]
 [-0.01653973 -0.02096177]
 [ 0.01046864 -0.05990141]]</td> 
  </tr>
  <tr>
    <td>b1</td>
    <td> [[ -1.02420756e-06]
 [  1.27373948e-05]
 [  8.32996807e-07]
 [ -3.20136836e-06]]</td> 
  </tr>
  <tr>
    <td>W2</td>
    <td> [[-0.01041081 -0.04463285  0.01758031  0.04747113]] </td> 
  </tr>
  <tr>
    <td>b2</td>
    <td> [[ 0.00010457]] </td> 
  </tr>
</table>  

### 4.4 - 4.1, 4.2 ve 4.3 parçalarını nn_model() içinde entegre edin ####

**Soru**: Sinir ağı modelinizi `nn_model()` içinde oluşturun.

**Talimatlar**: Sinir ağı modeli, önceki fonksiyonları doğru sırada kullanmalıdır.

In [ ]:
# GRADED FUNCTION: nn_model

def nn_model(X, Y, n_h, num_iterations=10000, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialize parameters, then retrieve W1, b1, W2, b2. Inputs: "n_x, n_h, n_y". Outputs = "W1, b1, W2, b2, parameters".
    ### START CODE HERE ### (≈ 5 lines of code)
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    ### END CODE HERE ###
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):
         
        ### START CODE HERE ### (≈ 4 lines of code)
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A2, Y, parameters)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads)
        
        ### END CODE HERE ###
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))

    return parameters

In [ ]:
X_assess, Y_assess = nn_model_test_case()

parameters = nn_model(X_assess, Y_assess, 4, num_iterations=10000, print_cost=False)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Beklenen Çıktı**:

<table style="width:90%">
  <tr>
    <td>W1</td>
    <td> [[-4.18494056  5.33220609]
 [-7.52989382  1.24306181]
 [-4.1929459   5.32632331]
 [ 7.52983719 -1.24309422]]</td> 
  </tr>
  <tr>
    <td>b1</td>
    <td> [[ 2.32926819]
 [ 3.79458998]
 [ 2.33002577]
 [-3.79468846]]</td> 
  </tr>
  <tr>
    <td>W2</td>
    <td> [[-6033.83672146 -6008.12980822 -6033.10095287  6008.06637269]] </td> 
  </tr>
  <tr>
    <td>b2</td>
    <td> [[-52.66607724]] </td> 
  </tr>
</table>  

### 4.5 Tahminler

**Soru**: Tahmin() oluşturarak tahmin yapmak için modelinizi kullanın.
Sonuçları tahmin etmek için ileri yayılımı kullanın.

**Hatırlatma**

predictions = $y_{prediction} = \mathbb 1 \text{{activation > 0.5}} = \begin{cases}
1 & \text{if}\ activation > 0.5 \\
0 & \text{otherwise}
\end{cases}$  
    
Örnek olarak, bir eşiğe dayalı olarak X matrisinin girişlerini 0 ve 1 olarak ayarlamak isterseniz şunları yaparsınız: ```X_new = (X > threshold)```

In [ ]:
# GRADED FUNCTION: predict

def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### (≈ 2 lines of code)
    A2, cache = forward_propagation(X, parameters)
    predictions = np.round(A2)
    ### END CODE HERE ###
    
    return predictions

In [ ]:
parameters, X_assess = predict_test_case()

predictions = predict(parameters, X_assess)
print("predictions mean = " + str(np.mean(predictions)))

**Beklenen Çıktı**: 


<table style="width:40%">
  <tr>
    <td>predictions mean</td>
    <td> 0.666666666667 </td> 
  </tr>
  
</table>

Modeli çalıştırmanın ve düzlemsel bir veri kümesinde nasıl performans gösterdiğini görmenin zamanı geldi. Modelinizi $n_h$ gizli birimlerinden oluşan tek bir gizli katmanla test etmek için aşağıdaki kodu çalıştırın.

In [ ]:
# Build a model with a n_h-dimensional hidden layer
parameters = nn_model(X, Y, n_h = 4, num_iterations=10000, print_cost=True)

# Plot the decision boundary
plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y)
plt.title("Decision Boundary for hidden layer size " + str(4))

**Beklenen Çıktı**:

<table style="width:40%">
  <tr>
    <td>Cost after iteration 9000</td>
    <td> 0.218607 </td> 
  </tr>
</table>

In [ ]:
# Print accuracy
predictions = predict(parameters, X)
print ('Accuracy: %d' % float((np.dot(Y, predictions.T) + np.dot(1 - Y, 1 - predictions.T)) / float(Y.size) * 100) + '%')

**Beklenen Çıktı**: 

<table style="width:15%">
  <tr>
    <td>Accuracy</td>
    <td> 90% </td> 
  </tr>
</table>

Doğruluk, Lojistik Regresyona kıyasla gerçekten yüksektir. Model, çiçeğin yaprak desenlerini öğrendi! Sinir ağları, lojistik regresyondan farklı olarak, oldukça doğrusal olmayan karar sınırlarını bile öğrenebilir.

Şimdi birkaç gizli katman boyutunu deneyelim.

### 4.6 - Gizli katman boyutunu ayarlama  ###

Aşağıdaki kodu çalıştırın. 1-2 dakika sürebilir. Çeşitli gizli katman boyutları için modelin farklı davranışlarını gözlemleyeceksiniz.

In [ ]:
# This may take about 2 minutes to run
plt.figure(figsize=(16, 32))
hidden_layer_sizes = [1, 2, 3, 4, 5, 20, 50]

for i, n_h in enumerate(hidden_layer_sizes):
    plt.subplot(5, 2, i + 1)
    plt.title('Hidden Layer of size %d' % n_h)

    parameters = nn_model(X, Y, n_h, num_iterations=5000)

    plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y)

    predictions = predict(parameters, X)
    
    accuracy = float((np.dot(Y, predictions.T) + np.dot(1 - Y, 1 - predictions.T)) / float(Y.size) * 100)
    print ("Accuracy for {} hidden units: {} %".format(n_h, accuracy))

**Tercüme**:
- Daha büyük modeller (daha fazla gizli birim içeren), eğitim kümesinde aşırı uyuma sebep olabilir.
- En iyi gizli katman boyutu n_h = 5 civarında görünüyor. Gerçekten de, buradaki bir değer, gözle görülür bir aşırı uyum sağlamadan verilere iyi uyuyor gibi görünüyor.
- Daha sonra, çok büyük modelleri (n_h = 50 gibi) aşıyı uyuma sebebiyet vermeden düzenlileştirme (regularization) hakkında da bilgi edineceksiniz.

**İsteğe bağlı sorular**:

Dilerseniz keşfedebileceğiniz bazı isteğe bağlı/not verilmeyen sorular:
- Bir sigmoid aktivasyonu veya bir ReLU aktivasyonu için tanh aktivasyonunu değiştirdiğinizde ne olur?
- Learning_rate ile oynayın. Ne oluyor?
- Ya veri setini değiştirirsek? (Aşağıdaki 5. bölüme bakın!)

<font color='blue'>
**Şunu yapmayı öğrendiniz:**

- Gizli bir katmanla eksiksiz bir sinir ağı oluşturma.
- Doğrusal olmayan bir birimi iyi kullanıldı.
- İleri yayılım ve geri yayılım uygulandı ve bir sinir ağı eğitildi.
- Fazla takma da dahil olmak üzere gizli katman boyutunu değiştirmenin etkisi görüldü.

## 5) Diğer veri kümelerindeki performans

İsterseniz, aşağıdaki veri kümelerinin her biri için tüm çalışma dosyasını yeniden çalıştırabilirsiniz.

In [ ]:
# Datasets
noisy_circles, noisy_moons, blobs, gaussian_quantiles, no_structure = load_extra_datasets()

datasets = {"noisy_circles": noisy_circles,
            "noisy_moons": noisy_moons,
            "blobs": blobs,
            "gaussian_quantiles": gaussian_quantiles}

### START CODE HERE ### (choose your dataset)
dataset = "noisy_moons"
### END CODE HERE ###

X, Y = datasets[dataset]
X, Y = X.T, Y.reshape(1, Y.shape[0])

# make blobs binary
if dataset == "blobs":
    Y = Y % 2

# Visualize the data
plt.scatter(X[0, :], X[1, :], c=Y, s=40, cmap=plt.cm.Spectral);

Referanslar:
- http://scs.ryerson.ca/~aharley/neural-networks/
- http://cs231n.github.io/neural-networks-case-study/